In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import random # we don't fix seed here so can use to re sample test patients

Global seed set to 0


In [2]:
# if want to change global seed to sth else
seed = 0
settings.seed = seed

Global seed set to 0


### Read, prep and train/test split

In [ ]:
# rna.h5ad is full dataset, change to rna_test.h5ad for a smaller subset
RNA = sc.read('data/TICAtlas.h5ad')
adata.obs['patient'] = adata.obs['patient'].astype("category")
adata.obs['gender'] = adata.obs['gender'].astype("category")
adata.obs['subtype'] = adata.obs['subtype'].astype("category")
adata.obs['cell_type'] = adata.obs['cell_type'].astype("category")
adata.obs['source'] = adata.obs['source'].astype("category")
number_of_test_patients = 15
patients = list(adata.obs['patient'].cat.categories)
test_patient_ids = random.sample(patients, number_of_test_patients)
adata[adata.obs.patient.isin(test_patient_ids)].obs[['subtype', 'patient']].drop_duplicates().groupby('subtype').count() # check class distribution for test set
adata_test = adata[adata.obs.patient.isin(test_patient_ids)].copy()
adata = adata[~adata.obs.patient.isin(test_patient_ids)].copy()
adata_test.obs.split = ["test"]
adata.obs.split = ["train"]
assert (len(set(adata.obs.subtype.unique()).intersection(adata_test.obs.subtype.unique())) == len(adata_test.obs.subtype.unique()))
adata = adata.concatenate(adata_test)
sc.pp.highly_variable_genes(adata, n_top_genes=4000, inplace=True)
adata = adata[:,adata.var.highly_variable]
adata.raw.X[:,adata.var.highly_variable]
adata.rename_categories("batch", ["train", "test"])
adata.obs.split = adata.obs.batch 
adata.write("./data/TICatlas_prep.h5ad")              
              